In [1]:
#https://towardsdatascience.com/k-means-clustering-chardonnay-reviews-using-scikit-learn-nltk-9df3c59527f3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\surab\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [3]:
df = pd.read_csv('TEST1.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer
0,"Beer geek, and love to bake",4,8,1,2,10,10,1,2,8,2,2
1,"I code, trek and hike",4,7,1,2,5,8,1,2,11,3,1
2,"Cooking, foodie, baker",2,6,3,1,5,10,1,2,5,2,2
3,"rap, hip-hop, music, guitar",3,7,1,2,3,4,1,2,8,2,2
4,"coffee, hungry, singer 😀",3,2,1,1,7,1,2,1,8,2,1
5,"social-media, twitter, facebook, vape",2,1,2,1,7,6,1,1,9,1,1
6,"friendly alcoholaholic, driver",2,5,3,2,2,3,1,1,2,2,1
7,"math major, integration, calculus, algebra",2,7,3,2,1,1,1,2,3,3,2
8,"creator, photshop",2,2,1,2,10,8,1,1,6,2,2
9,"pop culture, friendly geek, weed",2,5,2,2,2,8,2,1,3,1,1


In [4]:
# #create a list of stop words
# stop_words = set(stopwords.words("english"))
# #show how many words are in the list of stop words
# # print(len(stop_words))
# #179
# #construct a new list to store the cleaned text
# clean_desc = []

# def clean_text(text):
#     text = text.lower()
#     print(text)
#     string=''

#     text=re.sub(r"[-()\"#!@$%^&*{}?.,:]"," ", str(text))
#     text=re.sub(r"\s+"," ", str(text))
#     text=re.sub('[A-Aa-z0-9]+', ' ', str(text))

#     for word in text.split():
#         if word not in stop_words:
#             string += WordNetLemmatizer().lemmatize(word)+ ' '
#         return string

In [5]:
# df['Bios']=df['Bios'].apply(clean_text)

In [6]:
def remove_punct(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct
df['Bios'] = df['Bios'].apply(lambda x: remove_punct(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer
0,Beer geek and love to bake,4,8,1,2,10,10,1,2,8,2,2
1,I code trek and hike,4,7,1,2,5,8,1,2,11,3,1
2,Cooking foodie baker,2,6,3,1,5,10,1,2,5,2,2
3,rap hiphop music guitar,3,7,1,2,3,4,1,2,8,2,2
4,coffee hungry singer 😀,3,2,1,1,7,1,2,1,8,2,1


In [7]:
tokenizer = RegexpTokenizer(r'\w+')
df['Bios'] = df['Bios'].apply(lambda x: tokenizer.tokenize(x.lower()))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer
0,"[beer, geek, and, love, to, bake]",4,8,1,2,10,10,1,2,8,2,2
1,"[i, code, trek, and, hike]",4,7,1,2,5,8,1,2,11,3,1
2,"[cooking, foodie, baker]",2,6,3,1,5,10,1,2,5,2,2
3,"[rap, hiphop, music, guitar]",3,7,1,2,3,4,1,2,8,2,2
4,"[coffee, hungry, singer]",3,2,1,1,7,1,2,1,8,2,1


In [8]:
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [9]:
df['Bios'] = df['Bios'].apply(lambda x: remove_stopwords(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer
0,"[beer, geek, love, bake]",4,8,1,2,10,10,1,2,8,2,2
1,"[code, trek, hike]",4,7,1,2,5,8,1,2,11,3,1
2,"[cooking, foodie, baker]",2,6,3,1,5,10,1,2,5,2,2
3,"[rap, hiphop, music, guitar]",3,7,1,2,3,4,1,2,8,2,2
4,"[coffee, hungry, singer]",3,2,1,1,7,1,2,1,8,2,1


In [10]:
lemmatizer = WordNetLemmatizer()

def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

In [11]:
df['Bios'] = df['Bios'].apply(lambda x: word_lemmatizer(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer
0,"[beer, geek, love, bake]",4,8,1,2,10,10,1,2,8,2,2
1,"[code, trek, hike]",4,7,1,2,5,8,1,2,11,3,1
2,"[cooking, foodie, baker]",2,6,3,1,5,10,1,2,5,2,2
3,"[rap, hiphop, music, guitar]",3,7,1,2,3,4,1,2,8,2,2
4,"[coffee, hungry, singer]",3,2,1,1,7,1,2,1,8,2,1


In [12]:
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

df['Bios'] = df['Bios'].apply(lambda x: word_stemmer(x))
df.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer
0,beer geek love bake,4,8,1,2,10,10,1,2,8,2,2
1,code trek hike,4,7,1,2,5,8,1,2,11,3,1
2,cook foodi baker,2,6,3,1,5,10,1,2,5,2,2
3,rap hiphop music guitar,3,7,1,2,3,4,1,2,8,2,2
4,coffe hungri singer,3,2,1,1,7,1,2,1,8,2,1


In [13]:
# #create a list of stop words
stop_words = set(stopwords.words("english"))
# #show how many words are in the list of stop words
# print(len(stop_words))
# #179
# #construct a new list to store the cleaned text
# clean_desc = []
# for w in range(len(df.Bios)):
#     desc = df['Bios'].str.lower

#     #remove punctuation
#     desc = re.sub('[^a-zA-Z]', ' ', str(desc))
    
#     #remove tags
#     desc = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
#     #remove special characters and digits
#     desc = re.sub("(\\d|\\W)+"," ",desc)

#     split_text = desc.split()

#     #Lemmatisation
#     lem = WordNetLemmatizer()
#     split_text = [lem.lemmatize(word) for word in split_text if not word in stop_words and len(word) >2] 

#     split_text = " ".join(split_text)
#     clean_desc.append(split_text)

In [14]:
df

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer
0,beer geek love bake,4,8,1,2,10,10,1,2,8,2,2
1,code trek hike,4,7,1,2,5,8,1,2,11,3,1
2,cook foodi baker,2,6,3,1,5,10,1,2,5,2,2
3,rap hiphop music guitar,3,7,1,2,3,4,1,2,8,2,2
4,coffe hungri singer,3,2,1,1,7,1,2,1,8,2,1
5,socialmedia twitter facebook vape,2,1,2,1,7,6,1,1,9,1,1
6,friendli alcoholahol driver,2,5,3,2,2,3,1,1,2,2,1
7,math major integr calculu algebra,2,7,3,2,1,1,1,2,3,3,2
8,creator photshop,2,2,1,2,10,8,1,1,6,2,2
9,pop cultur friendli geek weed,2,5,2,2,2,8,2,1,3,1,1


In [15]:
#TF-IDF vectorizer
tfv = TfidfVectorizer(stop_words = "english",ngram_range = (1,1))
#transform
vec_text = tfv.fit_transform(df.Bios)
#returns a list of words.
words = tfv.get_feature_names()
print(words)
# tfidf = TfidfVectorizer(
#     min_df = 5,
#     max_df = 0.95,
#     max_features = 8000,
#     stop_words = 'english'
# )
# tfidf.fit(df['Bios'])
# text = tfidf.transform(df['Bios'])

# def get_top_keywords(data, clusters, labels, n_terms):
#     df = pd.DataFrame(data.todense()).groupby(clusters).mean()
    
#     for i,r in df.iterrows():
#         print('\nCluster {}'.format(i))
#         print(','.join([labels[t] for t in np.argsort(r)[-n_terms:]]))
            
# get_top_keywords(text, clusters, tfidf.get_feature_names(), 10)

['account', 'addict', 'alcoholahol', 'algebra', 'artist', 'bake', 'baker', 'basketbal', 'bball', 'beer', 'bike', 'busi', 'calculu', 'cartoon', 'channel', 'checkout', 'code', 'coffe', 'compsci', 'computersci', 'cook', 'creator', 'cultur', 'disney', 'dog', 'driver', 'druggi', 'enthusiast', 'facebook', 'foodi', 'footbal', 'freak', 'friend', 'friendli', 'game', 'gamer', 'garden', 'geek', 'guitar', 'gym', 'hike', 'hiphop', 'hungri', 'hype', 'instagram', 'integr', 'introvert', 'learn', 'let', 'life', 'like', 'live', 'look', 'love', 'lover', 'major', 'math', 'mathemat', 'meet', 'music', 'musician', 'nerd', 'new', 'peopl', 'photshop', 'piano', 'plant', 'playerfooood', 'pop', 'problemsolv', 'rap', 'read', 'roommat', 'sing', 'singer', 'soccer', 'socialmedia', 'togeth', 'travel', 'trek', 'tv', 'twitter', 'vape', 'web', 'weed', 'woohoo', 'writer', 'youtub']


In [16]:
#setup kmeans clustering
kmeans = KMeans(n_clusters = 6, n_init = 17, tol = 0.01, max_iter = 200)
#fit the data 
kmeans.fit(vec_text)
#this loop transforms the numbers back into words
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(common_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))
    

0 : love, coffe, instagram, bake, travel, dog, creator, photshop, plant, garden
1 : introvert, gamer, nerd, youtub, freak, dog, driver, druggi, enthusiast, facebook
2 : twitter, account, vape, facebook, socialmedia, footbal, cultur, disney, dog, driver
3 : look, problemsolv, roommat, friend, footbal, youtub, foodi, disney, dog, driver
4 : geek, compsci, mathemat, cultur, pop, weed, friendli, let, togeth, game
5 : major, busi, soccer, computersci, calculu, algebra, integr, math, druggi, enthusiast


In [21]:
from sklearn.cluster import MiniBatchKMeans, KMeans
vec= TfidfVectorizer(stop_words="english", ngram_range=(1,1))
vec.fit(df.Bios.values)
features = vec.transform(df.Bios.values)
clust1Bios = KMeans(init = 'k-means++', n_clusters = 5, n_init=10)
clust1Bios.fit(features)

KMeans(n_clusters=5)

In [22]:
yhat = clust1Bios.predict(features)
df['BioClusters'] = clust1Bios.labels_
df

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,BioClusters
0,beer geek love bake,4,8,1,2,10,10,1,2,8,2,2,0
1,code trek hike,4,7,1,2,5,8,1,2,11,3,1,2
2,cook foodi baker,2,6,3,1,5,10,1,2,5,2,2,2
3,rap hiphop music guitar,3,7,1,2,3,4,1,2,8,2,2,2
4,coffe hungri singer,3,2,1,1,7,1,2,1,8,2,1,0
5,socialmedia twitter facebook vape,2,1,2,1,7,6,1,1,9,1,1,2
6,friendli alcoholahol driver,2,5,3,2,2,3,1,1,2,2,1,4
7,math major integr calculu algebra,2,7,3,2,1,1,1,2,3,3,2,1
8,creator photshop,2,2,1,2,10,8,1,1,6,2,2,0
9,pop cultur friendli geek weed,2,5,2,2,2,8,2,1,3,1,1,4


In [23]:
Clust1Bios=df[df['BioClusters']==0]
Clust1Bios.head()

,Bios,Year,Major_Dept,Ambition,Goals,Music,Sports,Friday_Nights,School_balance,Movie_Genres,Social_Style,Transfer,BioClusters
0,beer geek love bake,4,8,1,2,10,10,1,2,8,2,2,0
4,coffe hungri singer,3,2,1,1,7,1,2,1,8,2,1,0
8,creator photshop,2,2,1,2,10,8,1,1,6,2,2,0
12,tv love disney cartoon,4,8,3,2,8,9,2,1,5,2,2,0
14,travel musician artist,3,4,1,1,7,10,2,3,1,3,2,0


In [24]:
Clust1wBios=df[df['BioClusters']==0]
Clust1wBios.to_csv('BioClusters/0.csv')
Clust1wBios=df[df['BioClusters']==1]
Clust1wBios.to_csv('BioClusters/1.csv')
Clust1wBios=df[df['BioClusters']==2]
Clust1wBios.to_csv('BioClusters/2.csv')
Clust1wBios=df[df['BioClusters']==3]
Clust1wBios.to_csv('BioClusters/3.csv')
Clust1wBios=df[df['BioClusters']==4]
Clust1wBios.to_csv('BioClusters/4.csv')